In [1]:
pip install numpy matplotlib tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [ ]:
pip install kaggle


In [ ]:

!kaggle datasets download -d anandkumarsahu09/cattle-breeds-dataset

In [7]:
!tar -xf cattle-breeds-dataset.zip


In [8]:
datagen = ImageDataGenerator(
    rescale=1./255,          # normalize pixel values
    rotation_range=20,       # rotate images randomly
    width_shift_range=0.1,   # shift horizontally
    height_shift_range=0.1,  # shift vertically
    zoom_range=0.2,          # random zoomS
    horizontal_flip=True,    # flip horizontally
    validation_split=0.2     # 20% data for validation
)

# Training data
train_data = datagen.flow_from_directory(
    'cattle Breeds/',               # path to dataset folder
    target_size=(224,224),   # resize images
    batch_size=32,
    class_mode='categorical', # multi-class classification
    subset='training'
)

# Validation data
val_data = datagen.flow_from_directory(
    'cattle Breeds/',
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

Found 969 images belonging to 5 classes.
Found 239 images belonging to 5 classes.


In [9]:
print("Training samples:", train_data.samples)
print("Validation samples:", val_data.samples)
num_classes = train_data.num_classes
print("Number of breeds:", num_classes)

Training samples: 969
Validation samples: 239
Number of breeds: 5


In [10]:


model = Sequential([

    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(num_classes, activation='softmax')
])

model.compile(
    optimizer=Adam(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


c:\Users\mural\OneDrive\Desktop\classification_proj\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,605 (42.61 MB)

 Trainable params: 11,169,605 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
!pip install scipy


  Using cached scipy-1.16.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached scipy-1.16.3-cp312-cp312-win_amd64.whl (38.6 MB)


In [13]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=20,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 32s 987ms/step - accuracy: 0.2549 - loss: 1.9160 - val_accuracy: 0.3766 - val_loss: 1.4711
Epoch 2/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 469ms/step - accuracy: 0.3715 - loss: 1.4443 - val_accuracy: 0.4184 - val_loss: 1.3216
Epoch 3/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 471ms/step - accuracy: 0.4623 - loss: 1.3419 - val_accuracy: 0.4937 - val_loss: 1.2297
Epoch 4/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 498ms/step - accuracy: 0.5108 - loss: 1.2284 - val_accuracy: 0.5105 - val_loss: 1.1909
Epoch 5/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 505ms/step - accuracy: 0.5366 - loss: 1.1844 - val_accuracy: 0.5397 - val_loss: 1.1866
Epoch 6/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 496ms/step - accuracy: 0.5273 - loss: 1.1817 - val_accuracy: 0.5439 - val_loss: 1.1691
Epoch 7/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 15s 486ms/step - accuracy: 0.5418 - loss: 1.1269 - val_accuracy: 0.5523 - val_loss: 1.2039
Epoch 8/20
31/31 ━━━━━━━━━━━━━━━━━━━━ 16s 508ms/step - accuracy: 0.5666 - loss: 1.0868 - val_accu

In [14]:
val_loss, val_acc = model.evaluate(val_data)
print(f"Validation Accuracy: {val_acc*100:.2f}%")

model.save("cow_breed_cnn.h5")
print("Model saved as cow_breed_cnn.keras")

8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - accuracy: 0.4979 - loss: 1.1505


Validation Accuracy: 49.79%
Model saved as cow_breed_cnn.keras
